In [7]:
import snowflake.connector
import json
import pandas as pd

class snowflake_helper:
    def __init__(self, data):
        self.create_connection(data)

    def create_connection(self, data):
    # To understna myconnection.toml file
    # refer https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect#connecting-using-the-connections-toml-file
        self.conn = snowflake.connector.connect(
                                                user='XXXXXXXXXX',
                                                password='XXXXXXX',
                                                account='rftahuh-wob23599',
                                                database = data.get('database_name','amazonebooks'),
                                                warehouse = data.get('warehouse','COMPUTE_WH'),
                                                schema = data.get('schema','PUBLIC')
                                                )
        self.curr = self.conn.cursor()

    def query_exec(self,query):
        self.curr.execute(query)

    def query_exec_getresult(self,query):
        result_df = pd.read_sql(query, self.conn) # reading in Pandas Data frame
        return result_df

    def connection_close(self):
        self.conn.close()

    def __del__(self):
        self.conn.close()

In [23]:
data = {
    "database_name":"DEV_DB",
    "table_name":"amazone_books",
    "data_location_dict":"",
    "stage_table":"amazone_books_stage"
}

database_name = data.get("database_name","")
warehouse = data.get("warehouse","COMPUTE_WH")
schema = data.get("schema","PUBLIC")
table_name = data.get("table_name", None)
data_location_dict = data.get("data_location_dict", None)
stage_table = data.get("stage_table",f'{table_name}_stage' )

snowflake_db = snowflake_helper({'database_name' : database_name,'warehouse' : warehouse,'schema': schema})
path = f'file://Data/20240227-amazone-books.csv'

query=f'''PUT '{path}' @{stage_table};'''
snowflake_db.query_exec(query)

snowflake_db.connection_close()

In [ ]:
df = pd.read_csv("D:/Learning/Snowflake/DataData/20240227-amazone-books.csv")